[Dataset link](https://www.kaggle.com/CooperUnion/cardataset)

[Reference](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92)

# Introduction to Word2Vec

Word2vec is one of the most popular technique to learn word embeddings using a two-layer neural network. Its input is a text corpus and its output is a set of vectors. Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space. For instance, the words women, men, and human might cluster in one corner, while yellow, red and blue cluster together in another.

There are two main training algorithms for word2vec, one is the continuous bag of words(CBOW), another is called skip-gram. The major difference between these two methods is that CBOW is using context to predict a target word while skip-gram is using a word to predict a target context. Generally, the skip-gram method can have a better performance compared with CBOW method, for it can capture two semantics for a single word. For instance, it will have two vector representations for Apple, one for the company and another for the fruit. For more details about the word2vec algorithm, please check [here](https://arxiv.org/pdf/1301.3781.pdf).

# Gensim Python Library Introduction

Gensim is an open source python library for natural language processing and it was developed and is maintained by the Czech natural language processing researcher Radim Řehůřek. Gensim library will enable us to develop word embeddings by training our own word2vec models on a custom corpus either with CBOW of skip-grams algorithms.

In [1]:
!pip install --upgrade gensim

# Download the data

## Dataset Description

This vehicle dataset includes features such as make, model, year, engine, and other properties of the car. We will use these features to generate the word embeddings for each make model and then compare the similarities between different make model.

In [2]:
!wget https://raw.githubusercontent.com/PICT-NLP/BE-NLP-Elective/main/2-Embeddings/data.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


# Implementation of Word Embedding with Gensim

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


## Data Preprocessing

Since the purpose of this tutorial is to learn how to generate word embeddings using genism library, we will not do the EDA and feature selection for the word2vec model for the sake of simplicity.

Genism word2vec requires that a format of ‘list of lists’ for training where every document is contained in a list and every list contains lists of tokens of that document. At first, we need to generate a format of ‘list of lists’ for training the make model word embedding. To be more specific, each make model is contained in a list and every list contains lists of features of that make model.

To achieve this, we need to do the following things

Create a new column for Make Model

In [5]:
df['Maker_Model']= df['Make']+ " " + df['Model']

Generate a format of ‘ list of lists’ for each Make Model with the following features: Engine Fuel Type, Transmission Type, Driven_Wheels, Market Category, Vehicle Size, Vehicle Style.

In [6]:
df1 = df[['Engine Fuel Type','Transmission Type','Driven_Wheels','Market Category','Vehicle Size', 'Vehicle Style', 'Maker_Model']]

df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)

df_clean = pd.DataFrame({'clean': df2})

sent = [row.split(',') for row in df_clean['clean']]

## Genism word2vec Model Training

We can train the genism word2vec model with our own custom corpus as following:

In [7]:
from gensim.models.word2vec import Word2Vec

Let’s try to understand the hyperparameters of this model.

1. `vector_size`: The number of dimensions of the embeddings and the default is 100.

2. `window`: The maximum distance between a target word and words around the target word. The default window is 5.

3. `min_count`: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.

4. `workers`: The number of partitions during training and the default workers is 3.

5. `sg`: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.

After training the word2vec model, we can obtain the word embedding directly from the training model as following.

In [8]:
model = Word2Vec(sent, min_count=1,vector_size= 50,workers=3, window =3, sg = 1)

Save the model

In [9]:
model.save("word2vec.model")

Load the model

In [10]:
model = Word2Vec.load("word2vec.model")

After training the word2vec model, we can obtain the word embedding directly from the training model as following.

In [11]:
model.wv['Toyota Camry']

array([ 0.00060954,  0.10690276,  0.05986022, -0.11626566, -0.05881133,
       -0.18526934,  0.00709567,  0.27551118, -0.10320179, -0.0604336 ,
        0.01600162, -0.01016074,  0.13940156, -0.02224435, -0.08011921,
        0.18529609,  0.15381187,  0.28613517, -0.11510384, -0.25761494,
       -0.07403318, -0.04513855,  0.24798553,  0.06555474,  0.14035505,
        0.00971497, -0.03118841,  0.32835603, -0.03530207, -0.00381901,
        0.0038671 ,  0.03232258,  0.0193231 , -0.01718037,  0.12143018,
       -0.10916604,  0.16562675, -0.08725581, -0.0136482 ,  0.05917208,
        0.11594134, -0.05374424, -0.18197812,  0.13060941,  0.32692796,
       -0.00556416, -0.03042129, -0.16028203, -0.02597837,  0.02022796],
      dtype=float32)

In [12]:
sims = model.wv.most_similar('Toyota Camry', topn=10)
sims

[('Mazda 6', 0.9814350008964539),
 ('Nissan Altima', 0.97865229845047),
 ('Dodge Dynasty', 0.9780991673469543),
 ('Suzuki Aerio', 0.9769200682640076),
 ('Chevrolet Cruze', 0.9751349091529846),
 ('Pontiac Grand Am', 0.973942220211029),
 ('Ford Windstar', 0.9735517501831055),
 ('Kia Optima', 0.972167432308197),
 ('Oldsmobile Eighty-Eight Royale', 0.9713041186332703),
 ('Oldsmobile Cutlass Ciera', 0.9701311588287354)]